<center><h1>Escenarios experimentales Internetworking</h1></center>

Presentado por: Holger Fernando Catucuamba Nepas <br>
Fecha: 23/02/2023

##  Importación de librerias necesarias

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Cargar el Dataset

In [2]:
datapath = 'https://raw.githubusercontent.com/hfcatucuamba/proyectoInternetworking/main/Escenario1.csv'
dataframe = pd.read_csv(datapath,low_memory=False, sep=';')

In [3]:
dataframe.head()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0,45.977.011.494.253,435,0,...,0,0,0,0,0,0,0,0,0,nonTOR
1,10.0.2.15,53913,216.58.208.46,80,6,259,0,77.220.077.220.077,259,0,...,0,0,0,0,0,0,0,0,0,nonTOR
2,10.0.2.15,53913,216.58.208.46,80,6,891,0,22.446.689.113.356,891,0,...,0,0,0,0,0,0,0,0,0,nonTOR
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0,18.621.973.929.237,1074,0,...,0,0,0,0,0,0,0,0,0,nonTOR
4,10.0.2.15,53913,216.58.208.46,80,6,315,0,63.492.063.492.064,315,0,...,0,0,0,0,0,0,0,0,0,nonTOR


## Normalizar los datos

In [4]:
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df

In [5]:
print (dataframe.shape)
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()
print(dataframe.describe())
print (list(dataframe))

(7000, 29)
        Source Port   Destination Port   Protocol   Flow Duration  \
count   7000.000000        7000.000000     7000.0    7.000000e+03   
mean   42153.076571        7647.390714        6.0    4.002204e+06   
std    17081.978193       17168.713987        0.0    4.450485e+06   
min       80.000000          21.000000        6.0    1.700000e+01   
25%    38894.000000         443.000000        6.0    2.013598e+05   
50%    45337.000000         443.000000        6.0    4.409195e+05   
75%    54698.000000         443.000000        6.0    9.829612e+06   
max    61156.000000       65514.000000        6.0    1.000000e+07   

        Flow IAT Max   Flow IAT Min   Fwd IAT Max   Fwd IAT Min   Bwd IAT Max  \
count   7.000000e+03   7.000000e+03  7.000000e+03  7.000000e+03  7.000000e+03   
mean    1.148187e+06   1.023332e+05  1.079414e+06  1.035305e+05  8.884348e+05   
std     2.113319e+06   4.321051e+05  2.135818e+06  6.952060e+05  1.933856e+06   
min     1.700000e+01  -4.340000e+02  0.0000

In [6]:
keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys) - 1]].copy()
x_normalised = dfNormalize(data_to_process)
print(x_normalised.describe())

        Protocol   Flow Duration   Flow Bytes/s   Flow Packets/s  \
count     7000.0     7000.000000    7000.000000      7000.000000   
mean         0.0        0.400219       0.000143         0.000167   
std          0.0        0.445049       0.011952         0.012051   
min          0.0        0.000000       0.000000         0.000000   
25%          0.0        0.020134       0.000000         0.000000   
50%          0.0        0.044090       0.000000         0.000000   
75%          0.0        0.982961       0.000000         0.000000   
max          0.0        1.000000       1.000000         1.000000   

        Flow IAT Mean   Flow IAT Std   Flow IAT Max   Flow IAT Min  \
count     7000.000000         7000.0    7000.000000    7000.000000   
mean         0.017407            0.0       0.114853       0.010376   
std          0.055836            0.0       0.211398       0.043627   
min          0.000000            0.0       0.000000       0.000000   
25%          0.000000            0.0 


## Obtener los datos para entrenar y de prueba

In [7]:
change_labels = lambda x: 1 if x == 'nonTOR' else 0

In [8]:
y_normalised = dataframe['label'].apply(change_labels)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x_normalised,
                                            y_normalised, test_size=0.3, random_state=42)

## Crear Modelo 

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Conv1D(64, 3, input_shape=(X_train.shape[1], 1), padding="same", activation="relu", kernel_constraint=MaxNorm(3)),
    Dropout(0.3),
    Conv1D(64, 3, padding="same", activation="relu", kernel_constraint=MaxNorm(3)),
    Flatten(),
    Dense(512, activation="relu", kernel_constraint=MaxNorm(3)),
    Dropout(0.3),
    Dense(10, activation="softmax")
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy", 
              metrics=["sparse_categorical_accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 24, 64)            256       
                                                                 
 dropout (Dropout)           (None, 24, 64)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 24, 64)            12352     
                                                                 
 flatten (Flatten)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5

In [11]:
batch_size = 32
steps_per_epoch = (X_train.shape[0]//batch_size)
validation_steps = (x_normalised.shape[0] // batch_size)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

In [12]:
%%time

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=32)

Epoch 1/25
154/154 [==============================] - 2s 10ms/step - loss: 0.3210 - sparse_categorical_accuracy: 0.8912 - val_loss: 0.0970 - val_sparse_categorical_accuracy: 0.9690
Epoch 2/25
154/154 [==============================] - 1s 9ms/step - loss: 0.0919 - sparse_categorical_accuracy: 0.9737 - val_loss: 0.0434 - val_sparse_categorical_accuracy: 0.9886
Epoch 3/25
154/154 [==============================] - 1s 9ms/step - loss: 0.0655 - sparse_categorical_accuracy: 0.9822 - val_loss: 0.0369 - val_sparse_categorical_accuracy: 0.9910
Epoch 4/25
154/154 [==============================] - 1s 10ms/step - loss: 0.0613 - sparse_categorical_accuracy: 0.9827 - val_loss: 0.0332 - val_sparse_categorical_accuracy: 0.9919
Epoch 5/25
154/154 [==============================] - 1s 9ms/step - loss: 0.0560 - sparse_categorical_accuracy: 0.9837 - val_loss: 0.0351 - val_sparse_categorical_accuracy: 0.9919
Epoch 6/25
154/154 [==============================] - 1s 9ms/step - loss: 0.0618 - sparse_categori

In [13]:
loss, accuracy = model.evaluate(X_test,y_test,batch_size=batch_size)
print('test_loss: %.2f' % (loss * 100.0))  
print('test_accuracy: %.2f' % (accuracy * 100.0))

66/66 [==============================] - 0s 3ms/step - loss: 0.0229 - sparse_categorical_accuracy: 0.9948
test_loss: 2.29
test_accuracy: 99.48


In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
model_probs = model.predict(X_test,batch_size=batch_size,verbose=0)
# predict crisp classes for test set
model_classes = np.argmax(model_probs,axis=1)
# reduce to 1d array
model_probs = model_probs[:, 0]
 
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, model_classes)*100
# precision tp / (tp + fp)
precision = precision_score(y_test, model_classes, average='weighted')*100
# recall: tp / (tp + fn)
recall = recall_score(y_test, model_classes, average='weighted')*100
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, model_classes, average='weighted')*100

print('Accuracy: %.2f' % accuracy)
print('F1 score: %.2f' % f1)
print('Precision: %.2f' % precision)
print('Recall: %.2f' % recall)

Accuracy: 99.48
F1 score: 99.48
Precision: 99.48
Recall: 99.48
